# Create a LangChain AI Agent in Python with watsonx 

**Author**: Anna Gutowska 

In this tutorial, we will use the LangChain Python package to build an AI agent that utilizes its custom tools to return a URL directing to [NASA's Astronomy Picture of the Day](https://apod.nasa.gov/apod/astropix.html). 

An [artificial intelligence (AI) agent](https://www.ibm.com/think/topics/ai-agents), otherwise known as a [Large Language Model (LLM)](https://www.ibm.com/topics/large-language-models) agent, is a system that performs tasks on behalf of a user or another system by designing its own workflow and utilizing available tools.

# Overview of AI agents 
One of the most common modalities of agentic AI approaches are chatbots. However, agentic technology can encompass a wide range of functions. These include planning, problem-solving, interacting with external environments, and executing actions. These agents can be deployed to solve complex tasks in various enterprise contexts. From software design and IT automation, including customer service bots, to code-generation tools and conversational AI assistants, AI agents leverage the capability of LLMs to work step-by-step.

Key processes that make AI agents unique in their autonomy are: 

* **Goal initializaiton and planning**. Although AI agents are autonomous in their planning of future actions, they require goals and environments defined by humans.

* **Reasoning using available tools**. An AI agent’s plan of action is based on the information it perceives, or its percepts. Often, AI agents do not have the full knowledge base needed for tackling all subtasks within a complex goal. To remedy this, AI agents use their available tools. These tools can include external datasets, algorithms, search tools, APIs and even other agents. We can give agents instructions to first "think" and plan their response before deciding which tools to use and iteratively improving the response, if needed. This can be done by aligning the predefined prompt structure with a ReAct (Reasoning and Action) framework. The ReAct prompting technique essentially tells the agent to reason slowly and to display each "thought". The verbal reasoning produced gives insight into how responses are being formulated. On the contrary, planning is not a necessary step for an agent when working on simple tasks. Instead, an agent can iteratively reflect on its responses and improve them without planning its next steps.

* **Learning and reflection**. AI agents use feedback mechanisms, such as other AI agents and human-in-the-loop (HITL), to improve the accuracy of their responses. 

 Traditional LLMs, like the IBM [Granite Model](https://www.ibm.com/products/watsonx-ai/foundation-models) we will be using in this tutorial, produce their responses based on the data used to train them. There is a knowledge and reasoning limitation. This means that current information, such as today's date, is not available to the model without any additional tools. In contrast, agentic technology utilizes tool calling on the backend to obtain up-to-date information, optimize workflow, and create specific tasks autonomously to achieve complex goals. In this process, the autonomous agent is learning to adapt to user expectations over time, providing a  personalized experience and comprehensive responses. This tool calling can be achieved without human intervention and broadens the possibilities for real-world applications of these AI systems.

We encourage you to check out our [AI Agents article](https://www.ibm.com/think/topics/ai-agents) for more in-depth information on the various AI agent architectures and their abstractions. 

# Prerequisites

You need an [IBM Cloud account](https://cloud.ibm.com/registration?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-trial) to create a [watsonx.ai](https://www.ibm.com/products/watsonx-ai?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-product) project.

# Steps

## Step 1. Set up your environment

While you can choose from several tools, this tutorial walks you through how to set up an IBM account to use a Jupyter Notebook. Jupyter Notebooks are widely used within [data science](https://www.ibm.com/topics/data-science?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-tutorials-_-ibmcom) to combine code, text, images, and [data visualizations](https://www.ibm.com/topics/data-visualization?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-tutorials-_-ibmcom) to formulate a well-formed analysis.

1. Log in to [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-trial) using your IBM Cloud account.

2. Create a [watsonx.ai project](https://www.ibm.com/docs/en/watsonx/saas?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&topic=projects-creating-project&cm_sp=ibmdev-_-developer-tutorials-_-ibmcom).
	- Note the project ID in project > Manage > General > Project ID. You'll need this for the tutorial.

3. Create a [Jupyter Notebook](https://www.ibm.com/docs/en/watsonx/saas?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&topic=editor-creating-managing-notebooks&cm_sp=ibmdev-_-developer-tutorials-_-ibmcom).

This step will open a Notebook environment where you can copy the code from this tutorial to implement an AI agent of your own. Alternatively, you can download this notebook to your local system and upload it to your watsonx.ai project as an asset. This Jupyter Notebook is available on GitHub.

## Step 2. Set up a Watson Machine Learning service instance and API key

1. Create a [Watson Machine Learning](https://cloud.ibm.com/catalog/services/watson-machine-learning?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-tutorials-_-trial) service instance (choose the Lite plan, which is a free instance).

2. Generate an [API Key in WML](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&context=cpdaas). 
	- Can be found in Manage > Access (IAM) > API Keys (left hand panel) > Create +. Save this API key for use in this tutorial.


3. Associate the WML service to the project you created in [watsonx.ai](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/assoc-services.html?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&context=cpdaas). 
	- Can be found in Project > Services & Integrations > + Associate Service > Add "Watson Machine Learning"

## Step 3. Install and import relevant libraries and set up your credentials

We'll need a few libraries and modules for this tutorial. Make sure to import the ones below, and if they're not installed, you can resolve this with a quick pip install. LangChain will be the framework and developer toolkit used.

In [ ]:
#installations
%pip install pandas
%pip install langchain
%pip install langchain_ibm
%pip install langchain_core
%pip install IPython
%pip install nasapy

In [9]:
#imports
import pandas as pd
import nasapy

from langchain.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import render_text_description_and_args
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_ibm import WatsonxLLM
from datetime import datetime

Set up your credentials. Input your API key and project ID.

In [2]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "YOUR_API_KEY_HERE",   
    "project_id": "YOUR_PROJECT_ID_HERE"
}

Let's establish our connection with the NASA API that we will be using later in this activity. This API does not require authentication. With a small rate limit, you can get started using 'DEMO_KEY' as your key. To avoid the low rate limitations, you can [register for your own NASA API key](](https://api.nasa.gov/)) and replace it as the key value below. Registering for a personal key is quick, free, and simple. 

In [2]:
n = nasapy.Nasa(key='DEMO_KEY')

## Step 4. Initialize a basic agent with no tools

This step is important as it will produce a clear example of an agent's behavior with and without tools. Let's start by setting our parameters. 

The model parameters available can be found [here](https://ibm.github.io/watson-machine-learning-sdk/model.html). We experimented with various model parameters, including Temperature, Top P, and Top K. [Here's](https://www.ibm.com/docs/en/watsonx/saas?topic=lab-model-parameters-prompting) some more information on model parameters and what they mean. It is important to set our `stop_sequences` here in order to limit agent hallucination. 

In [3]:
param = {
    "decoding_method": "greedy",
    "temperature": 0, 
    "min_new_tokens": 5,
    "max_new_tokens": 250,
    "stop_sequences":['\nObservation', '\n\n']
    }

For this tutorial, we suggest using IBM's Granite 13B Chat model as the LLM to achieve similar results. You are free to use any AI model of choice. The purpose of these models in LLM applications is to serve as the reasoning engine that decides which actions to take.

In [29]:
model = WatsonxLLM(
	model_id =  "ibm/granite-13b-chat-v2",
	url = credentials.get("url"),
	apikey = credentials.get("apikey"),
	project_id =  credentials.get("project_id"),
	params = param
	)

We'll set up a prompt template in case you would like to ask multiple questions. 

In [20]:
template = "Answer the {query} accurately."
prompt = PromptTemplate.from_template(template)

And now we can set up a chain with our prompt and our LLM. This allows the generative model to produce a response.

In [31]:
agent = prompt | model

In the next step of this tutorial, we will be creating a tool that retrieves today's date. As we have covered, traditional LLMs cannot obtain the current date on their own. Let's verify this.

In [32]:
agent.invoke({"query": "What is today's date?"})

'\n\nA: Today is [display current date].\n\n'

Evidently, the LLM is unable to provide us with the current date. The training data used for this model contained information prior to today's date and without the appropriate tools, the agent does not have access to real-time information. 

## Step 5. Define the agent's tools

Unlike traditional LLMs, AI agents can provide more comprehensive responses to diverse tasks through their tool usage, memory, and planning. Let's define the two tools our agent will be using: 

* `get_todays_date()` - uses the Python `datetime` package to return today's date in YYYY-MM-DD format. 
* `get_astronomy_image()` - utilizes the NASA API to obtain the Astronomy Picture of the Day. After the tool acquires the image, its URL is returned. 

Note: The date variable is set to an empty string because it is used by both tools and is thus, a global variable. 

In [39]:
date = ''   

@tool
def get_todays_date():
    """Get today's date in YYYY-MM-DD format."""
    global date
    date = datetime.now().strftime("%Y-%m-%d")
    return date


@tool(return_direct=True)
def get_astronomy_image():
    """Get NASA's Astronomy Picture of the Day on given date."""
    global date
        
    apod = n.picture_of_the_day(date, hd=True)
    
    return apod['url']


tools = [get_todays_date, get_astronomy_image]

## Step 6. Generate a response with the AI agent

Next, we will set up a prompt template to ask multiple questions. This template tells the agent how to print its "thought process". This "thought process" refers to a view of the agent's subtasks, tools used, and final output. This prompt also instructs the agent to return its responses in JSON Blob format and to consider information it has stored in its memory. 

In [40]:
system_prompt = """Respond to the human as helpfully and accurately as possible. You have access to the following tools:
{tools}
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).
Valid "action" values: "Final Answer" or {tool_names}
Provide only ONE action per $JSON_BLOB, as shown:"
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
Follow this format:
Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
Begin! Reminder to ALWAYS respond with a valid json blob of a single action.
Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation"""

Below, we are telling the agent to display the intermediate steps it takes in the `agent_scratchpad` following the user input. 

In [41]:
human_prompt = """{input}
{agent_scratchpad}
(reminder to always respond in a JSON blob)"""

Here, we establish the order of human, agent, and tool messages. We create the template to feature the previously defined `system_prompt` followed by an optional list of messages collected in the agent's memory, if any, and finally, the `human_prompt` which includes both the `input` and `agent_scratchpad`. 

In [42]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
    ]
)

Now, lets finalize our prompt template by rendering the tool names, descriptions, and arguments in plain text. This allows the agent to access the information pertaining to each tool and its corresponding intended use case. This also means we can add and remove tools without needing to alter our entire prompt template. 

In [43]:
prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

An important feature of AI agents is their memory. Agents are able to store past conversations and past findings in their memory to improve the accuracy and relevance of their responses going forward. In our case, we will use LangChain's `ConversationBufferMemory()` as a means of memory storage. 

In [44]:
memory = ConversationBufferMemory()

And now we can set up a chain with our agent's scratchpad, memory, our prompt and our LLM. The `AgentExecutor` class is used to execute the agent. It takes the agent, its tools, error handling approach, verbose parameter, and memory. 

In [45]:
chain = ( RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt | model | JSONAgentOutputParser()                                               
)                           

agent_executor = AgentExecutor(agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory)

We are now able to ask the agent questions. Recall the agent's previous inability to provide us with the current date. Now that the agent has its tools available to use, let's try asking the same question again. 

In [46]:
agent_executor.invoke({"input": "What is today's date?"})



> Entering new AgentExecutor chain...


Action:
```
{
  "action": "get_todays_date",
  "action_input": {}
}
```

2024-07-09
Action:
```
{
  "action": "Final Answer",
  "action_input": "Today's date is 2024-07-09."
}
```

> Finished chain.


{'input': "What is today's date?",
 'history': '',
 'output': "Today's date is 2024-07-09."}

Great! The agent is now able to tell us the current date. As seen in the JSON Blob produced, the agent recognized that it should utilize the `get_todays_date` tool to reach this result. 

Now, let's try asking a more complex question.

In [47]:
agent_executor.invoke({"input": "Show me NASA's Astronomy Picture of the Day today."})



> Entering new AgentExecutor chain...

Action:
```
{
  "action": "get_todays_date",
  "action_input": {}
}
```
Observation2024-07-09
Action:
```
{
  "action": "get_astronomy_image",
  "action_input": {}
}
```
Observationhttps://apod.nasa.gov/apod/image/2407/NoctilucentFlorida_Pouquet_960.jpg


> Finished chain.


{'input': "Show me NASA's Astronomy Picture of the Day today.",
 'history': "Human: What is today's date?\nAI: Today's date is 2024-07-09.",
 'output': 'https://apod.nasa.gov/apod/image/2407/NoctilucentFlorida_Pouquet_960.jpg'}

Neat! The agent used both of its available tools to return a URL which leads to an image of today's Astronomy Picture of the Day via NASA's API. Check out the image [here](https://apod.nasa.gov/apod/image/2407/GianniTumino_Etna&MW_14mm_JPG_LOGO__1024pix.jpg "here"). Since we wanted to see the image  from today, the agent used the `get_todays_date` tool first in order to then use that date for the `get_astronomy_image` tool. Additionally, the agent is successfully updating its knowledge base as it learns new information and experiences new interactions as seen by the history output. 

And that's it!

# Summary and next steps

In this tutorial, you created a AI agent using LangChain in Python with watsonx. You created a tool to return today's date and another tool to return today's Astronomy Picture of the Day using NASA's open-source API.

The sample output is important as it shows the steps the agent took in creating its own agent workflow using available tools. In our case, the LLM on its own was not able to achieve the first subtask of the problem - finding the current date. Hence, the tools granted to the agent were incredibly useful and vital for achieving the goal. 

We encourage you to check out the [LangChain documentation page](https://python.langchain.com/v0.2/docs/tutorials/agents/) for more information and tutorials on AI agents. 


## Try wantsonx for free

Build an AI strategy for your business on one collaborative AI and data platform called IBM [watsonx](https://www.ibm.com/watsonx?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-product), which brings together new generative AI capabilities, powered by foundation models, and traditional machine learning into a powerful platform spanning the AI lifecycle. With [watsonx.ai](https://www.ibm.com/products/watsonx-ai?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-product), you can train, validate, tune, and deploy models with ease and build AI applications in a fraction of the time with a fraction of the data.

Try [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx&cm_sp=ibmdev-_-developer-_-trial), the next-generation studio for AI builders.

## Next steps

Explore [more articles and tutorials about watsonx](https://developer.ibm.com/components/watsonx/?utm_source=ibm_developer&utm_content=in_content_link&utm_id=tutorials_awb-create-langchain-rag-system-python-watsonx) on IBM Developer.